# Практика построения регрессии
### Вопрос 1
Давайте проанализируем данные опроса 4361 женщин из Ботсваны:
О каждой из них мы знаем:

 * сколько детей она родила (признак ceb)
 * возраст (age)
 * длительность получения образования (educ)
 * религиозная принадлежность (religion)
 * идеальное, по её мнению, количество детей в семье (idlnchld)
 * была ли она когда-нибудь замужем (evermarr)
 * возраст первого замужества (agefm)
 * длительность получения образования мужем (heduc)
 * знает ли она о методах контрацепции (knowmeth)
 * использует ли она методы контрацепции (usemeth)
 * живёт ли она в городе (urban)
 * есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [71]:
import pandas as pd
import numpy as np

In [72]:
df = pd.read_csv('botswana.tsv', sep = '\t')

In [73]:
df.religion.unique()

array(['catholic', 'protestant', 'spirit', 'other'], dtype=object)

### Вопрос 2
Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [74]:
df.shape

(4361, 15)

In [75]:
df.dropna().shape

(1834, 15)

### Вопрос 3 
В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так: 
 * создать новый бинарный признак $x2 = 1, x1 = 'не применимо'$ и $x2=0$ иначе 
 * заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида
$$y=β0+β1x1+β2x2,$$
на тех объектах, где x1 было измерено, регрессионное уравнение примет вид
$$y=β0+β1x,$$
а там, где x1 было "не применимо", получится
$$y=β0+β1c+β2.$$
Выбор c влияет только на значение и интерпретацию β2, но не β1.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

1. Создайте признак nevermarr, равный единице там, где в agefm пропуски.
2. Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
3. Замените NaN в признаке agefm на cagefm=0.
4. У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
5. Сколько осталось пропущенных значений в признаке heduc?

In [76]:
df['nevermarr'] = map(lambda x: 1 if pd.isnull(x) else 0, df.agefm)

In [77]:
df.drop('evermarr', axis = 1, inplace=True)

In [78]:
cagefm = 0

In [79]:
df['agefm'] = df['agefm'].fillna(cagefm)

In [80]:
cheduc1 = -1

In [81]:
df['heduc'] = map(lambda x, y: cheduc1 if (pd.isnull(x) and (y == 1)) else x, df.heduc, df.nevermarr)

In [82]:
len(filter(pd.isnull, df.heduc))

123

### Вопрос 4
Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: 
 * создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), 
 * замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [83]:
df['idlnchld_noans'] = map(lambda x: 1 if pd.isnull(x) else 0, df.idlnchld)
df['heduc_noans'] = map(lambda x: 1 if pd.isnull(x) else 0, df.heduc)
df['usemeth_noans'] = map(lambda x: 1 if pd.isnull(x) else 0, df.usemeth)

In [84]:
cidlnchld = -1
cheduc2 = -2
cusemeth = -1

In [85]:
df['idlnchld'] = df.idlnchld.fillna(cidlnchld)
df['heduc'] = df.heduc.fillna(cheduc2)
df['usemeth'] = df.usemeth.fillna(cusemeth)

In [86]:
df = df.dropna()

In [87]:
df.shape

(4348, 18)

In [88]:
df.shape[0]*df.shape[1]

78264

### Вопрос 5
Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R2? Округлите до трёх знаков после десятичной точки.

In [89]:
import statsmodels.formula.api as smf

In [90]:
df.columns

Index([u'ceb', u'age', u'educ', u'religion', u'idlnchld', u'knowmeth',
       u'usemeth', u'agefm', u'heduc', u'urban', u'electric', u'radio', u'tv',
       u'bicycle', u'nevermarr', u'idlnchld_noans', u'heduc_noans',
       u'usemeth_noans'],
      dtype='object')

In [91]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + ' \
             'usemeth + agefm + heduc + urban + electric + radio + tv + ' \
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=df)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 23 Oct 2016   Prob (F-statistic):               0.00
Time:                        17:07:04   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

### Вопрос 7
Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [92]:
import statsmodels.stats.api as sms

In [93]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [94]:
m2 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + ' \
             'usemeth + agefm + heduc + urban + electric + radio + tv + ' \
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=df)
fitted = m2.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Sun, 23 Oct 2016   Prob (F-statistic):               0.00
Time:                        17:09:54   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

### Вопрос 8
Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [98]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + ' \
             'usemeth + agefm + heduc + urban + electric + ' \
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=df)
fitted = m3.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Sun, 23 Oct 2016   Prob (F-statistic):               0.00
Time:                        17:12:10   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [99]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [100]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + ' \
             'usemeth + agefm + heduc + urban + electric + ' \
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=df)
fitted = m4.fit(cov_type = 'HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 23 Oct 2016   Prob (F-statistic):               0.00
Time:                        17:12:40   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [105]:
print "F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m4.fit())

F=0.919236, p=0.467231, k1=5.000000


### Вопрос 9
Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [106]:
m5 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + ' \
             'agefm + heduc + urban + electric + ' \
             'bicycle + nevermarr + idlnchld_noans + heduc_noans', 
             data=df)
fitted = m5.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Sun, 23 Oct 2016   Prob (F-statistic):               0.00
Time:                        17:16:36   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.

In [107]:
print "F=%f, p=%f, k1=%f" % m4.fit().compare_f_test(m5.fit())

F=92.890582, p=0.000000, k1=2.000000


In [108]:
m4.fit().compare_f_test(m5.fit())

(92.890582301098021, 3.1552009480371243e-40, 2.0)

### Вопрос 10

Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

 * ~~Итоговая модель объясняет 63% вариации отклика~~
 * С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18])
 * ~~У женщин, не знающих, какое количество детей идеально, в среднем на 0.66 ребёнка больше (p=0.002, 95% доверительный интервал — [0.2, 1.1])~~
 * У женщин, никогда не выходивших замуж, при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9])
 * У женщин, не знающих, какое количество детей идеально, в среднем на $β_{idlnchldnoans}+c_{idlnchld}β_{idlnchld}≈0.58$ детей больше
 * ~~У женщин, знакомых с методами контрацепции, при прочих равных в среднем на 0.6 ребёнка меньше (p=0.001, 95% доверительный интервал для разницы между средними — [-0.9, -0.2])~~

In [109]:
m = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + ' \
             'usemeth + agefm + heduc + urban + electric + ' \
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=df)
fitted = m.fit(cov_type = 'HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 23 Oct 2016   Prob (F-statistic):               0.00
Time:                        17:18:00   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [111]:
 0.6565  + cidlnchld*0.0770

0.5795